In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SID
from emoji import UNICODE_EMOJI
import pandas as pd
import re

In [2]:
df = pd.read_json('../data/country_tweets.json')

(40000, 2)

In [3]:
# number of @USERs
df['at_count'] = df['raw_text'].apply( lambda txt: len(re.findall(r'@\w+', txt)) )

# number of #s
df['hash_count'] = df['raw_text'].apply( lambda txt: len(re.findall(r'\#\w+', txt)) )

# number of emoji
df['emoji_count'] = df['raw_text'].apply( lambda txt: len([char for char in txt if char in UNICODE_EMOJI]) )

# has url?
df['has_url'] = df['raw_text'].apply( lambda txt: 1 if len(re.findall(r'https://.+', txt)) > 0 else 0 )

# get rid of @USER, #tags, emoji, big spaces, and URLs --> clean_text
df['clean_text'] = df['raw_text'].apply( lambda txt: re.sub(r'@\w+', '', txt) )
df['clean_text'] = df['clean_text'].apply( lambda txt: re.sub(r'\#\w+', '', txt) )
df['clean_text'] = df['clean_text'].apply( lambda txt: re.sub(r'https://.+', '', txt) )
df['clean_text'] = df['clean_text'].apply( lambda txt: ''.join([char for char in txt if char not in UNICODE_EMOJI]) )
df['clean_text'] = df['clean_text'].apply( lambda txt:  re.sub(r'\s{2}', ' ', txt) )

# number of characters
df['text_len'] = df['clean_text'].apply( lambda txt: len(txt) )

# number of words
df['word_count'] = df['clean_text'].apply( lambda txt: len(txt.split(' ')) )

# sentiment of tweet
sid = SID()
df['sentiment'] = df['clean_text'].apply( lambda txt: sid.polarity_scores(txt)['compound'] )

df.head()

,country,raw_text,at_count,hash_count,emoji_count,has_url,clean_text,text_len,word_count,sentiment
0,ENGLAND,@HauwaTafida I wonder who asked them,1,0,0,0,I wonder who asked them,24,6,0.0000
1,ENGLAND,😩😩cute af,0,0,2,0,cute af,7,2,0.4588
2,ENGLAND,@KateDrummond_ @DIBSRescue I’m SO pleased for ...,2,0,2,0,I’m SO pleased for you guys!,30,8,0.6391
3,ENGLAND,@den2114 Sold her country out,1,0,0,0,Sold her country out,21,5,0.0000
4,ENGLAND,Why isn’t silence an option?,0,0,0,0,Why isn’t silence an option?,28,5,0.0000


In [4]:
df.describe()

,at_count,hash_count,emoji_count,has_url,text_len,word_count,sentiment
count,40000.000000,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.843875,0.23225,0.482050,0.368525,57.868650,11.969275,0.129390
std,1.255524,0.77645,1.485726,0.482411,35.468604,6.779608,0.408243
min,0.000000,0.00000,0.000000,0.000000,0.000000,1.000000,-0.975500
25%,0.000000,0.00000,0.000000,0.000000,27.000000,6.000000,0.000000
50%,1.000000,0.00000,0.000000,0.000000,52.000000,11.000000,0.000000
75%,1.000000,0.00000,0.000000,1.000000,89.000000,18.000000,0.449700
max,11.000000,13.00000,92.000000,1.000000,144.000000,34.000000,0.993700


In [5]:
df.to_json('../data/processed_tweets.json')